In [1]:
import cv2
import numpy as np

from collections import deque
from timeit import default_timer as timer

# Catch frame from webcam
camera = cv2.VideoCapture(1)

# Define variables for hight and width shape of the frames
HEIGH, WIDTH = 300, 400   

# Define objects boundaries size
MIN_OBJECT_AREA = 100
MAX_OBJECT_AREA = 10000

# Define the background
last_frame = np.zeros((HEIGH, WIDTH,3) , np.uint8)
background = np.zeros((HEIGH, WIDTH,3) , np.uint8)

# Initializing deque object for center points of the detected object
points = deque(maxlen=50)

# Restart timer for FPS
fps_start = timer()    
    
# Increasing FPS counter
counter_fps = 0

# Define FPS Variable
FPS = 0

########################################################################################################################
    
# Function return 3-Dimension frame
def expands_dimensions(frame):
    
    new_image = np.zeros((HEIGH, WIDTH, 3), np.uint8)
    new_image[:, :, 0] = frame
    new_image[:, :, 1] = frame
    new_image[:, :, 2] = frame
    
    return new_image

# Convert frame from rgb to gray
def gray_frame(frame_rgb):
    
    # Converting captured frame to GRAY by OpenCV function    
    gray_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    
    frame_gray = np.zeros(frame_rgb.shape, np.uint8)
    frame_gray[:,:,0] = gray_frame
    frame_gray[:,:,1] = gray_frame
    frame_gray[:,:,2] = gray_frame
    
    return frame_gray

# Function define the mask
def mask(frame_rgb, background):
    
    # Converting captured frame to GRAY by OpenCV function    
    frame_gray = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
    
    # Create one more frame with Gaussian blur
    frame_gray = cv2.GaussianBlur(frame_gray, (25, 25), 0)  

    # Converting captured frame to GRAY by OpenCV function        
    background = cv2.cvtColor(background, cv2.COLOR_BGR2GRAY)
    
    # Create one more frame with Gaussian blur
    background = cv2.GaussianBlur(background, (25, 25), 0)    
    
    # Return mask to detect change between two frames   
    abs_diff = cv2.absdiff(frame_gray, background)
    
    # Function exclude values that ara more than treshhold = 15 0 and more than 255
    _, mask = cv2.threshold(abs_diff, 20, 255, cv2.THRESH_BINARY)
    
    # Dilates the object in the frame
    dilated_mask = cv2.dilate(mask, None, iterations = 5) 
    
    return dilated_mask


# Function display the frames on the screen in one window
def display_windows(frame_rgb, frame_gray, frame_mask):   
        
    # Create left window    
    main_window = np.hstack((frame_rgb, frame_gray, frame_mask))

    # Plotting all the frames in one window
    cv2.imshow("Main_Window", main_window)     

# Function create 3 frames from the frame we read
def preproccess_frames(frame, last_frame):
    
    # Resize the main frame to (WIDTH, HEIGH) shape
    frame = cv2.resize(frame, (WIDTH, HEIGH))
        
    # Copy frame to work with deffrent variable
    frame_rgb = frame.copy()
    
    # Return gray frame in 3 dimension
    frame_gray = gray_frame(frame_rgb)
    
    # Return mask frame 
    frame_mask = mask(frame_rgb, last_frame)
    
    # Define last frame
    last_frame = frame_rgb.copy()
    
    return frame, frame_rgb, frame_gray, frame_mask, last_frame

# Function find all the relevant contours in the frame
def findContours(frame_rgb, frame_mask):
    
    # Function return array of all contours we found
    contours, _ = cv2.findContours(frame_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sorted the contours and define the larger first
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    
    # Scan the contours list
    for contour in contours:

        # Return square area of the given contour
        contour_area = cv2.contourArea(contour)

        # Find contours between MIN_OBJECT_AREA to MAX_OBJECT_AREA
        if contour_area < MAX_OBJECT_AREA:
            if MIN_OBJECT_AREA < contour_area:
                
                # Get an approximate rectangle coordinates
                (x_min, y_min, box_width, box_height) = cv2.boundingRect(contour)

                # Drawing rectangle on the frame
                frame_rgb = cv2.rectangle(frame_rgb, (x_min, y_min), (x_min +box_width, y_min +box_height), (0, 255, 0), 2)
                
                # Store the rectangle coordinates around the object
                rectangle = np.array([x_min, y_min, box_width, box_height])
            else:
                # Contour is a sorted list so all the rest items irrelevant
                break
    
    return frame_rgb

# Function manage the frames reader variables like efps etc'
def reader_manger(FPS, fps_start, counter_fps):
       
    # Variable says if keep reading frame or quit
    quit = False

    # Stopping the timer for FPS
    fps_stop = timer()

    # Print FPS every 1 second
    if 1.0 <= fps_stop - fps_start:

        # Define FPS
        FPS = counter_fps

        # Reset FPS counter
        counter_fps = 0

        # Restart timer for FPS
        fps_start = timer()       

    # Function waits for key to be pressed    
    key = cv2.waitKey(10) % 256

    # If 'n' is pressed, we catchs the frame and define it as the background
    if key == ord('n'):
        tracking_on = False

    # If 'q' key is pressed then quit from app
    if key == ord('q'):
        quit = True   

    return FPS, fps_start, counter_fps, quit

In [2]:
# Loop reading frame by frame and processing them
while True:
    
    # Increasing FPS counter
    counter_fps += 1
    
    # Capturing frames one-by-one from camera
    ret, frame = camera.read()

    # If the frame was not retrieved then we break the loop
    if not ret or frame is None:
        break

    # Function return 3 diffrent kind of frames
    frame, frame_rgb, frame_gray, frame_mask, last_frame = preproccess_frames(frame, last_frame)
    
    # Function find countors around the objects and return drawn frame
    frame_rgb = findContours(frame_rgb, frame_mask)

    # Expend mask dimension to 3 dimension
    frame_mask = expands_dimensions(frame_mask)        

    
    
    
    
    # Display all frames in one window
    display_windows(frame_rgb, frame_gray, frame_mask)

    # Function manage the frames reader variables
    FPS, fps_start, counter_fps, quit = reader_manger(FPS, fps_start, counter_fps)

    # If quit is true so we stop read frames
    if quit == True:
        break

# Releasing camera
camera.release()

# Destroying all opened OpenCV windows
cv2.destroyAllWindows()      